In [29]:
data_in = './data_in'
data_out = './data_out'
data_prefix = 'bois_doug_wash'

In [30]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']

In [31]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk

In [32]:
from nltk.corpus import stopwords

In [33]:
words = []
for x in stopwords.words('english'):
    words.append(x)

In [34]:
len(words)

179

In [35]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kaleb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kaleb\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaleb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\kaleb\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [36]:
book = open(data_in+'/pg35399.txt', 'r', encoding='utf-8-sig').readlines()

In [37]:
len(book)

3967

In [38]:
test = re.compile("^\s*CHAPTER\s+{}\s*$".format(roman))

NameError: name 'roman' is not defined

In [39]:
for x in book:
    print(re.search(test,x))

NameError: name 'test' is not defined

In [40]:
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
chap_pats = {
    23: {
        'start_line': 515,
        'end_line': 3736,
        'chapter': re.compile('^\s*CHAPTER\s+{}\s*$'.format(roman))
    },
    202: {
        'start_line': 788,
        'end_line': 12240,
        'chapter': re.compile("^\s*CHAPTER\s+{}.\s*".format(roman))
    },
    408: {
        'start_line': 55,
        'end_line': 6740,
        'chapter': re.compile("^\s*{}.\s*$".format(roman))
    },
    59116: {\
        'start_line': 90,
        'end_line': 1597,
        'chapter': re.compile("^\s*{}.\s*$".format(roman))
    },
    62799: {
        'start_line': 360,
        'end_line': 9147,
        'chapter': re.compile('^\s*CHAPTER\s+{}\s*$'.format(roman))
    },
    64504: {
        'start_line': 333,
        'end_line': 6380,
        'chapter': re.compile('^\s*CHAPTER\s+{}\s*$'.format(roman))
    },
    2376: {
        'start_line': 338,
        'end_line': 7075,
        'chapter': re.compile('^\s*Chapter\s+{}.\s*'.format(roman))
    },
    15210: {
        'start_line': 187,
        'end_line': 7455,
        'chapter': re.compile("^\s*{}\s*$".format(roman))
    },
    15359: {
        'start_line': 130,
        'end_line': 5683,
        'chapter': re.compile("^\s*{}\s+[A-Z]+".format(roman))
    },
    26507: {
        'start_line': 180,
        'end_line': 3794,
        'chapter': re.compile("^\s*CHAPTER\s+{}.\s*$".format(roman))
    },
    31839: {
        'start_line': 110,
        'end_line': 970,
        'chapter': re.compile('ADDRESS.')
    },
    35399: {
        'start_line': 165,
        'end_line': 3023,
        'chapter': re.compile("^\s*CHAPTER\s+{}\s*$".format(roman))
    },
    45125: {
        'start_line': 205,
        'end_line': 890,
        'chapter': re.compile("^\s*{}\s*$".format(roman))
    }
}

In [41]:
def acquire_epubs(epub_list, chap_pats, OHCO=OHCO):
    
    my_lib = []
    my_doc = []

    for epub_file in epub_list:
        
        # Get PG ID from filename
        book_id = int(epub_file.split('in')[1].split('.')[0].replace('pg', '').split('-')[0][1:])
        print("BOOK ID", book_id)
        
        # Import file as lines
        lines = open(epub_file, 'r', encoding='utf-8-sig').readlines()
        df = pd.DataFrame(lines, columns=['line_str'])
        df.index.name = 'line_num'
        df.line_str = df.line_str.str.strip()
        df['book_id'] = book_id
        
        # FIX CHARACTERS TO IMPROVE TOKENIZATION
        df.line_str = df.line_str.str.replace('—', ' — ')
        df.line_str = df.line_str.str.replace('-', ' - ')
        
        # Get book title and put into LIB table -- note problems, though
        book_title = re.sub(r"The Project Gutenberg eBook( of|,) ", "", df.loc[0].line_str, flags=re.IGNORECASE)
        book_title = re.sub(r"Project Gutenberg's ", "", book_title, flags=re.IGNORECASE)
        
        # Remove cruft
        a = chap_pats[book_id]['start_line'] - 1
        b = chap_pats[book_id]['end_line'] + 1
        df = df.iloc[a:b]
        
        # Chunk by chapter
        chap_lines = df.line_str.str.match(chap_pats[book_id]['chapter'])
        chap_nums = [i+1 for i in range(df.loc[chap_lines].shape[0])]
        df.loc[chap_lines, 'chap_num'] = chap_nums
        df.chap_num = df.chap_num.ffill()

        # Clean up
#         df = df[~df.chap_num.isna()] 
        df = df.dropna(subset=['chap_num']) # Remove everything before Chapter 1
        df = df.loc[~chap_lines] # Remove chapter heading lines
        df['chap_num'] = df['chap_num'].astype('int')
        
        # Group -- Note that we exclude the book level in the OHCO at this point
        df = df.groupby(OHCO[1:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string
        
        # Split into paragrpahs
        df = df['line_str'].str.split(r'\n\n+', expand=True).stack().to_frame().rename(columns={0:'para_str'})
        df.index.names = OHCO[1:3] # MAY NOT BE NECESSARY UNTIL THE END
        df['para_str'] = df['para_str'].str.replace(r'\n', ' ').str.strip()
        df = df[~df['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
        
        # Set index
        df['book_id'] = book_id
        df = df.reset_index().set_index(OHCO[:3])

        # Register
        my_lib.append((book_id, book_title, epub_file))
        my_doc.append(df)

    docs = pd.concat(my_doc)
    library = pd.DataFrame(my_lib, columns=['book_id', 'book_title', 'book_file']).set_index('book_id')
    return library, docs

In [42]:
epubs = [epub for epub in sorted(glob('{}/*.txt'.format(data_in)))]

In [43]:
# Import file as lines

In [93]:
LIB, DOC = acquire_epubs(epubs, chap_pats)

BOOK ID 202
BOOK ID 23
BOOK ID 408
BOOK ID 59116
BOOK ID 62799
BOOK ID 64504
BOOK ID 15210
BOOK ID 15359
BOOK ID 2376
BOOK ID 26507
BOOK ID 31839
BOOK ID 35399
BOOK ID 45125


In [94]:
LIB

,book_title,book_file
book_id,,
202,"Project Gutenberg’s My Bondage and My Freedom,...",./data_in\202-0.txt
23,"Narrative of the Life of Frederick Douglass, b...",./data_in\23-0.txt
408,"The Souls of Black Folk, by W. E. B. Du Bois",./data_in\408-0.txt
59116,"Why is the Negro Lynched?, by Frederick Douglass",./data_in\59116-0.txt
62799,"John Brown, by W. E. Burghardt Du Bois",./data_in\62799-0.txt
64504,"Working with the Hands, by Booker T.",./data_in\64504-0.txt
15210,"Darkwater, by W. E. B. Du Bois",./data_in\pg15210.txt
15359,"The Negro, by W.E.B. Du Bois",./data_in\pg15359.txt
2376,"Up From Slavery: An Autobiography, by Booker T...",./data_in\pg2376.txt


In [95]:
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):
    
    # Paragraphs to Sentences
    df = doc_df.para_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pd.Series(nltk.pos_tag(nltk.word_tokenize(x)))
        return s
            
    df = df.sent_str\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [96]:
TOKEN = tokenize(DOC, ws=True)

In [97]:
TOKEN

pos_tuple  pos token_str
book_id chap_num para_num sent_num token_num                               
202     1        0        0        0             (PLACE, NN)   NN     PLACE
                                   1               (OF, NNP)  NNP        OF
                                   2            (BIRTH, NNP)  NNP     BIRTH
                                   3                  (-, :)    :         -
                                   4                  (-, :)    :         -
...                                                      ...  ...       ...
45125   4        34       1        25         (brighter, NN)   NN  brighter
                                   26              (and, CC)   CC       and
                                   27            (more, RBR)  RBR      more
                                   28         (cheerful, JJ)   JJ  cheerful
                                   29            (view., NN)   NN     view.

[693647 rows x 3 columns]

In [98]:
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')

In [99]:
TOKEN

pos_tuple  pos token_str  \
book_id chap_num para_num sent_num token_num                                  
202     1        0        0        0             (PLACE, NN)   NN     PLACE   
                                   1               (OF, NNP)  NNP        OF   
                                   2            (BIRTH, NNP)  NNP     BIRTH   
                                   3                  (-, :)    :         -   
                                   4                  (-, :)    :         -   
...                                                      ...  ...       ...   
45125   4        34       1        25         (brighter, NN)   NN  brighter   
                                   26              (and, CC)   CC       and   
                                   27            (more, RBR)  RBR      more   
                                   28         (cheerful, JJ)   JJ  cheerful   
                                   29            (view., NN)   NN     view.   

                                              term_str  
book_id chap_num para_num sent_num token_num            
202     1        0        0        0             place  
                                   1                of  
                                   2             birth  
                                   3                    
                                   4                    
...                                                ...  
45125   4        34       1        25         brighter  
                                   26              and  
                                   27             more  
                                   28         cheerful  
                                   29             view  

[693647 rows x 4 columns]

In [226]:
VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [227]:
VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')

In [228]:
VOCAB

,term_str,n,num
term_id,,,
0,,10909,0
1,01,1,1
2,1,34,1
3,10,19,1
4,100,11,1
...,...,...,...
24209,à,1,0
24210,æschylus,2,0
24211,æsop,1,0


In [229]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [230]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [231]:
VOCAB[VOCAB.stop == 1].sample(10)

,term_str,n,num,stop
term_id,,,,
23621,were,3040,0,1
24174,yourself,29,0,1
11316,in,15202,0,1
21584,those,670,0,1
7054,down,528,0,1
13686,me,2385,0,1
15405,own,720,0,1
23448,was,8054,0,1
13284,m,25,0,1


In [232]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
VOCAB['p_stem'] = VOCAB.term_str.apply(stemmer.stem)

In [233]:
VOCAB.sample(10)

,term_str,n,num,stop,p_stem
term_id,,,,,
7376,editor,12,0,0,editor
21768,to,20108,0,1,to
20594,stripe,1,0,0,stripe
17013,provincial,2,0,0,provinci
8393,fade,8,0,0,fade
22981,vagrancy,6,0,0,vagranc
18208,rice,25,0,0,rice
16196,plaint,2,0,0,plaint
7489,elgar,1,0,0,elgar


In [234]:
LIB_author_title = []
for x in LIB['book_title']:
    LIB_author_title.append(x.split('by'))

LIB_author_title = pd.DataFrame(LIB_author_title)
LIB_author_title.columns = ['title', 'Author']
LIB_author_title.index = LIB.index
LIB = pd.concat([LIB, LIB_author_title], axis = 1)

In [235]:
DOC.to_csv('{}/{}-DOC.csv'.format(data_out, data_prefix))
LIB.to_csv('{}/{}-LIB.csv'.format(data_out, data_prefix))
VOCAB.to_csv('{}/{}-VOCAB.csv'.format(data_out, data_prefix))
TOKEN.to_csv('{}/{}-TOKEN.csv'.format(data_out, data_prefix))

In [283]:
DOC = pd.read_csv('{}/{}-DOC.csv'.format(data_out, data_prefix))
LIB = pd.read_csv('{}/{}-LIB.csv'.format(data_out, data_prefix))
VOCAB = pd.read_csv('{}/{}-VOCAB.csv'.format(data_out, data_prefix)).set_index('term_str')
TOKEN = pd.read_csv('{}/{}-TOKEN.csv'.format(data_out, data_prefix))

In [288]:
VOCAB = VOCAB[~VOCAB.index.isna()]

In [289]:
def kaleb_TFIDF(token, vocab, ohco, tftype):
    bow = token.groupby(ohco+['term_str']).term_str.count().to_frame('tf_n')
    D = bow.groupby(ohco).tf_n
    if tftype == 'tf_cp':
        bow['tf_cp'] = D.apply(lambda x: x / x.sum())
    elif tftype == 'tf_bool':
        bow['tf_bool'] = 1
    elif tftype == 'tf_l2':
    # bow['tf_jp'] = D.apply(lambda x: x / x.sum().sum()) # jp = P(w,d) 
        bow['tf_l2'] = D.apply(lambda x: x / np.sqrt((x**2).sum()))
    elif tftype == 'tf_logn':
        bow['tf_logn'] = D.apply(lambda x: np.log2(1 + x))
    elif tftype == 'tf_sub':
        bow['tf_sub'] = D.apply(lambda x: 1 + np.log2(x))
    elif tftype == 'tf_max4':
        bow['tf_max4'] = D.apply(lambda x: .4 + .6 * (x / x.max()))
    elif tftype == 'tf_max5':
        bow['tf_max5'] = D.apply(lambda x: .5 + .5 * (x / x.max()))
    elif tftype == 'tf_max9':
        bow['tf_max9'] = D.apply(lambda x: .9 + .1 * (x / x.max()))
    elif tftype == 'tf_bool2':
        bow['tf_bool2'] = D.apply(lambda x: 1 / len(x))
    else:
        print(" ")

    # df computation
    vocab['df'] = bow.groupby('term_str').tf_n.count()
    # idf computation
    N_docs = len(D.groups)
    vocab['idf'] = np.log2(N_docs/(vocab.df))
    vocab['dfidf'] = vocab.df*vocab.idf
    
#     bow['dfidf']
    
    
#     for tf_type in bow.columns:
    bow[f'tfidf_{tftype[2:]}'] = bow[tftype]*vocab.idf
    
    tfidf_sum_cols = []
#     for tf_type in bow.columns[:10]:
    bow_col = f"tfidf_{tftype[2:]}"
    vocab_col = f"tfidf{tftype[2:]}_sum"
    tfidf_sum_cols.append(vocab_col)
    vocab[vocab_col] = bow.groupby('term_str')[bow_col].sum()
#         vocab[vocab_col] = normalize(vocab[vocab_col], N_docs)
    
    return bow, vocab

# TFIDF

In [239]:
def bow_tf_calculator(token, ohco, tftype):
    bow = token.groupby(ohco+['term_str']).term_str.count().to_frame('tf_n')
    D = bow.groupby(ohco).tf_n
    if tftype == 'tf_cp':
        bow['tf_cp'] = D.apply(lambda x: x / x.sum())
    elif tftype == 'tf_bool':
        bow['tf_bool'] = 1
    elif tftype == 'tf_l2':
    # bow['tf_jp'] = D.apply(lambda x: x / x.sum().sum()) # jp = P(w,d) 
        bow['tf_l2'] = D.apply(lambda x: x / np.sqrt((x**2).sum()))
    elif tftype == 'tf_logn':
        bow['tf_logn'] = D.apply(lambda x: np.log2(1 + x))
    elif tftype == 'tf_sub':
        bow['tf_sub'] = D.apply(lambda x: 1 + np.log2(x))
    elif tftype == 'tf_max4':
        bow['tf_max4'] = D.apply(lambda x: .4 + .6 * (x / x.max()))
    elif tftype == 'tf_max5':
        bow['tf_max5'] = D.apply(lambda x: .5 + .5 * (x / x.max()))
    elif tftype == 'tf_max9':
        bow['tf_max9'] = D.apply(lambda x: .9 + .1 * (x / x.max()))
    elif tftype == 'tf_bool2':
        bow['tf_bool2'] = D.apply(lambda x: 1 / len(x))
    else:
        print(" ")
    
    return D, bow
  

In [240]:
def vocab_df_calculator(vocab, bow, D):
    vocab['df'] = bow.groupby('term_str').tf_n.count()
    # idf computation
    N_docs = len(D.groups)
    vocab['idf'] = np.log2(N_docs/(vocab.df))
    vocab['dfidf'] = vocab.df*vocab.idf
    
    return vocab
    

In [333]:
len(D.groups)

5908

In [341]:
vocab.sort_values('idf').head(50)

,term_id,n,num,stop,p_stem,df,idf,dfidf,tfidf_max4_sum
term_str,,,,,,,,,
the,21438,48419,0,1,the,5144,0.199779,1027.664120,954.024885
of,14982,27909,0,1,of,4846,0.285875,1385.352292,1029.075147
and,1482,27126,0,1,and,4791,0.302343,1448.525374,1063.593429
to,21768,20108,0,1,to,4345,0.443314,1926.197805,1289.493074
in,11316,15202,0,1,in,4247,0.476226,2022.530622,1251.021058
a,742,13603,0,1,a,3975,0.571715,2272.566989,1386.510858
that,21428,8223,0,1,that,3126,0.918352,2870.768501,1586.561429
for,9030,5687,0,1,for,2728,1.114826,3041.245824,1595.294160
it,12066,5791,0,1,it,2613,1.176963,3075.403539,1639.447114


In [337]:
vocab.sort_values('tfidf_max4_sum', ascending=False).head(50)

,term_id,n,num,stop,p_stem,df,idf,dfidf,tfidf_max4_sum
term_str,,,,,,,,,
i,11043,9022,0,1,i,2155,1.454982,3135.486117,2055.624336
he,10404,5279,0,1,he,1678,1.815927,3047.125691,1795.238954
was,23448,8054,0,1,wa,2518,1.230392,3098.125905,1776.418411
is,12037,5587,0,1,is,2230,1.405626,3134.546239,1736.554042
his,10691,4524,0,1,hi,1683,1.811635,3048.981115,1692.375050
it,12066,5791,0,1,it,2613,1.176963,3075.403539,1639.447114
as,1886,5240,0,1,as,2488,1.247683,3104.236151,1626.719533
my,14431,3608,0,1,my,1299,2.185268,2838.663646,1610.722049
be,2518,3743,0,1,be,2024,1.545461,3128.012125,1604.198167


In [223]:
vocab.loc[:,'nenne'] = bow.groupby('term_str').tf_n.count()

In [260]:
OHCO[:3]

['book_id', 'chap_num', 'para_num']

In [ ]:
vocab.sort_values(by='dfidf', ascending=False).head(50)

In [243]:
D, bow = bow_tf_calculator(TOKEN, OHCO[:3], 'tf_max4')

In [245]:
vocab =vocab_df_calculator(VOCAB, bow, D)

In [265]:
bow.head(2)

tf_n   tf_max4
book_id chap_num para_num term_str                
23      1        1        1835         1  0.446154
                          a            5  0.630769

In [269]:
vocab =vocab.iloc[1:, :]

In [270]:
bow['tf_max4']*vocab.idf

book_id  chap_num  para_num  term_str 
23       1         1         1835         3.938653
                             a            0.360620
                             about        1.488070
                             accurate     4.436327
                             age          2.773483
                                            ...   
64504    19        48        to           0.443314
                             vouchsafe    6.566095
                             which        1.069164
                             who          1.315838
                             wide         3.496809
Length: 425272, dtype: float64

In [121]:
bow_tf_calculator(TOKEN, OHCO[:3], 'tf_max4')

tf_n   tf_max4
book_id chap_num para_num term_str                 
23      1        1                      7  0.723077
                          1835          1  0.446154
                          a             5  0.630769
                          about         2  0.492308
                          accurate      1  0.446154
...                                   ...       ...
64504   19       48       to            3  1.000000
                          vouchsafe     1  0.600000
                          which         1  0.600000
                          who           1  0.600000
                          wide          1  0.600000

[428319 rows x 2 columns]

In [328]:
OHCO[:3]

['book_id', 'chap_num', 'para_num']

In [342]:
bow, vocab  = kaleb_TFIDF(TOKEN, VOCAB, OHCO[:2], 'tf_max4')

<ipython-input-289-210457c47e0b>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vocab['df'] = bow.groupby('term_str').tf_n.count()
<ipython-input-289-210457c47e0b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vocab['idf'] = np.log2(N_docs/(vocab.df))
<ipython-input-289-210457c47e0b>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

### Remove the na value in the index(the first row in vocab table)

In [343]:
bow

tf_n   tf_max4  tfidf__max4
book_id chap_num term_str                             
23      1        1835         1  0.406122     1.628545
                 a           50  0.706122     0.000000
                 about        7  0.442857     0.017873
                 absent       1  0.406122     1.078802
                 accurate     1  0.406122     1.866111
...                         ...       ...          ...
64504   19       yet          1  0.401935     0.165487
                 york         4  0.407742     0.638197
                 you          1  0.401935     0.209689
                 your         3  0.405806     0.443322
                 youth        1  0.401935     0.746788

[184024 rows x 3 columns]

In [292]:
vocab

,term_id,n,num,stop,p_stem,df,idf,dfidf,tfidf_max4_sum
term_str,,,,,,,,,
01,1,1,1,0,01,1,12.528454,12.528454,5.512520
1,2,34,1,0,1,34,7.440991,252.993703,165.414588
10,3,19,1,0,10,18,8.358529,150.453524,97.814195
100,4,11,1,0,100,9,9.358529,84.226762,46.037774
1000,5,8,1,0,1000,8,9.528454,76.227633,40.416029
...,...,...,...,...,...,...,...,...,...
à,24209,1,0,0,à,1,12.528454,12.528454,5.637804
æschylus,24210,2,0,0,æschylu,2,11.528454,23.056908,10.643234
æsop,24211,1,0,0,æsop,1,12.528454,12.528454,5.261951


In [344]:
TFIDF = bow.tfidf__max4.unstack(fill_value=0)

In [345]:
OHCO[1]

'chap_num'

In [346]:
TFIDF = TFIDF.groupby(OHCO[1]).mean()

In [347]:
TFIDF

term_str,01,1,10,100,1000,10000,100000,1000000,10000000,100000000,...,zulu,zululand,zulus,zunis,zymbabwe,à,æschylus,æsop,æsthetic,élite
chap_num,,,,,,,,,,,,,,,,,,,,,
1,0.00000,0.089046,0.000000,0.137084,0.000000,0.000000,0.000000,0.000000,0.151612,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.173959,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.00000,0.287564,0.338775,0.000000,0.000000,0.000000,0.173079,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.173218,0.000000,0.206821,0.000000,0.000000,0.000000
4,0.00000,0.095617,0.000000,0.000000,0.307229,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240137,0.000000
5,0.00000,0.229367,0.135308,0.000000,0.000000,0.000000,0.207618,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.290167
6,0.00000,0.115128,0.000000,0.175530,0.184905,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.250175,0.000000,0.000000,0.000000,0.209693,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.00000,0.114853,0.135533,0.000000,0.000000,0.224496,0.000000,0.000000,0.000000,0.000000,...,0.252836,0.496524,0.455952,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.00000,0.127491,0.302274,0.389606,0.204596,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.249122,0.000000,0.231066,0.000000,0.000000,0.319694,0.000000,0.000000
9,0.00000,0.286549,0.338209,0.219210,0.230359,0.280493,0.260037,0.000000,0.243876,0.310238,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.360068,0.000000,0.000000,0.000000,0.000000


In [348]:
sum_method = 'tfidf_max4_sum'
top_4000 = vocab.sort_values(sum_method, ascending=False)[:4000]

In [349]:
sum_method = 'dfidf'
top_10 = vocab.sort_values(sum_method, ascending=False)[:10]

In [350]:
TFIDF[top_4000.index]

term_str,students,africa,john,brown,tuskegee,plantation,coloured,crime,labor,cotton,...,influenced,hosts,convictions,counter,bitterly,sufferings,conquering,confused,scarred,veiled
chap_num,,,,,,,,,,,,,,,,,,,,,
1,0.098878,0.522023,0.278814,0.234945,0.106832,0.170779,0.110328,0.181429,0.187460,0.140853,...,0.119664,0.477168,0.119001,0.119664,0.000000,0.000000,0.119301,0.119691,0.119301,0.000000
2,0.107475,0.316630,0.107013,0.237325,0.117817,0.234272,0.241411,0.244792,0.252239,0.253333,...,0.129118,0.000000,0.128898,0.000000,0.129040,0.000000,0.129118,0.258194,0.000000,0.128898
3,0.276283,0.110425,0.199137,0.181754,0.233472,0.228561,0.118431,0.243353,0.257822,0.256409,...,0.128909,0.000000,0.128909,0.000000,0.129119,0.000000,0.258126,0.129119,0.000000,0.129012
4,0.278000,0.150812,0.299436,0.190205,0.179487,0.241747,0.180577,0.197856,0.249440,0.354433,...,0.000000,0.000000,0.128836,0.129079,0.000000,0.129765,0.000000,0.000000,0.000000,0.257954
5,0.276030,0.120787,0.184961,0.147498,0.217156,0.168378,0.288408,0.057172,0.238990,0.121307,...,0.154988,0.154988,0.000000,0.000000,0.154783,0.154633,0.000000,0.000000,0.000000,0.000000
6,0.204789,0.305119,0.122068,0.215719,0.209937,0.113968,0.282626,0.287069,0.238476,0.182573,...,0.309814,0.000000,0.155443,0.309591,0.000000,0.155116,0.155249,0.309754,0.000000,0.000000
7,0.196256,0.248582,0.123437,0.226450,0.214414,0.273534,0.216600,0.118564,0.178502,0.247516,...,0.000000,0.000000,0.310850,0.154593,0.000000,0.154800,0.155199,0.000000,0.309561,0.310042
8,0.222469,0.206259,0.200840,0.330355,0.318651,0.247935,0.239233,0.320507,0.133494,0.275461,...,0.000000,0.343672,0.000000,0.171890,0.000000,0.000000,0.172099,0.171975,0.171890,0.172099
9,0.243765,0.155179,0.154388,0.281142,0.185996,0.278862,0.178552,0.146231,0.373341,0.075575,...,0.000000,0.194052,0.000000,0.000000,0.193475,0.000000,0.000000,0.000000,0.194628,0.387465


In [351]:
bow.loc[list(top_10.index),:]

,,,tf_n,tf_max4,tfidf__max4
book_id,chap_num,term_str,,,


In [352]:
list(top_10.index)

['support',
 'bad',
 'proper',
 'account',
 'direction',
 'cost',
 'prejudice',
 'generally',
 'faith',
 'son']

In [327]:
VOCAB.sort_values('tfidf_max4_sum', ascending=False).iloc[50:100, :]

,term_id,n,num,stop,p_stem,df,idf,dfidf,tfidf_max4_sum
term_str,,,,,,,,,
time,21720,1335,0,0,time,1006,2.554040,2569.363758,1229.169921
our,15253,1403,0,1,our,830,2.831487,2350.133865,1224.589307
some,19920,1285,0,1,some,985,2.584474,2545.707083,1221.770552
than,21417,1260,0,1,than,956,2.627587,2511.973461,1204.698053
will,23800,1282,0,1,will,817,2.854262,2331.931925,1201.100121
its,12085,1328,0,1,it,901,2.713071,2444.476804,1195.709091
what,23645,1118,0,1,what,856,2.786987,2385.660978,1172.499477
if,11084,1217,0,1,if,888,2.734038,2427.825961,1169.001530
white,23725,1302,0,0,white,836,2.821095,2358.435402,1165.057718
